In [5]:
import torch
import torch.nn as nn
import cv2, os
!pip install -q torch_snippets torch_summary
from torch_snippets import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.5 MB/s eta 0:00:00


In [7]:
if os.path.exists('Faceswap-Deepfake-Pytorch'):
  !wget -q  https://www.dropbox.com/s/5ji7jl7httso9ny/person_images.zip
  !wget -q https://raw.githubusercontent.com/sizhky/deep-fake-util/main/random_warp.py
  !unzip -q person_images.zip

!wget -O random_warp.py https://raw.githubusercontent.com/sizhky/deep-fake-util/main/random_warp.py

import random_warp
from random_warp import get_training_data

--2025-03-12 17:13:23--  https://raw.githubusercontent.com/sizhky/deep-fake-util/main/random_warp.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4163 (4.1K) [text/plain]
Saving to: ‘random_warp.py’

random_warp.py      100%[===================>]   4.07K  --.-KB/s    in 0s      

2025-03-12 17:13:24 (45.0 MB/s) - ‘random_warp.py’ saved [4163/4163]



In [9]:
import cv2, os
# Update the path below to the actual location of your haarcascade_frontalface_default.xml file
haarcascade_path = "/usr/local/lib/python3.10/dist-packages/cv2/data/haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(haarcascade_path)

In [10]:
#@ For Cropping face in image:
def crop_face(img):
  gray=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  faces=face_cascade.detectMultiScale(gray, 1.3, 5)
  if(len(faces)>0):
    for (x, y, w, h) in faces:
      img2=img[y:(y+h), x:(x+w), :]
    img2=cv2.resize(img2, (256, 256))
    return img2, True
  else:
    return img, False

In [11]:
!mkdir cropped_faces_personA
!mkdir cropped_faces_personB

def crop_images(folder):
  images=Glob(folder + '/*.jpg')
  for i in range(len(images)):
    img=read(images[i], 1)
    img2, face_detected=crop_face(img)
    if(face_detected==False):
      continue
    else:
      cv2.imwrite('cropped_faces_' + folder + '/' + str(i) + '.jpg', cv2.cvtColor(img2, cv2.COLOR_RGB2BGR))

crop_images('personA')
crop_images('personB')

In [15]:
from torch.utils.data import Dataset, DataLoader
device='cuda' if torch.cuda.is_available() else 'cpu'
class ImageDataset(Dataset):
  def __init__(self, items_A, items_B):
    self.items_A = np.concatenate([read(f, 1)[None] for f in items_A])/255.
    self.items_B=np.concatenate([read(f,1)[None] for f in items_B])/255.
    self.items_A += self.items_B.mean(axis=(0,1, 2)) - self.items_A.mean(axis=(0,1, 2))

  def __len__(self):
    return min(len(self.items_A), len(self.items_B))

  def __getitem__(self, ix):
    a, b = choose(self.items_A), choose(self.items_B)
    return a, b

  def collate_fn(self, batch):
    imsA, imsB=list(zip(*batch))
    imsA, targetA= get_training_data(imsA, len(imsA))
    imsB, targetB=get_training_data(imsB, len(imsB))
    imsA, imsB, targetA, targetB = [torch.Tensor(i).permute(0, 3, 1, 2).to(device) for i in [imsA, imsB, targetA, targetB]]
    return imsA, imsB, targetA, targetB


In [19]:
a = ImageDataset(Glob('cropped_faces_personA'), Glob('cropped_faces_personB'))
x = DataLoader(a, batch_size=32, collate_fn=a.collate_fn)

ValueError: need at least one array to concatenate

In [17]:
import inspect
inspect(*next(iter(x)))
for i in next(iter(x)):
 subplots(i[:8], nc=4, sz=(4,2))

NameError: name 'x' is not defined

In [14]:
#@ Helper Function:
def _ConvLayer(in_features, out_features):
  return nn.Sequential(
      nn.Conv2d(in_features, out_features,
                kernel_size=5, stride=2, padding=2),
      nn.LeakyReLU(0.1, inplace=True)
  )

def _UpScale(in_features, out_features):
  return nn.Sequential(
      nn.ConvTranspose2d(in_features, out_features,
                         kernel_size=2, stride=2, padding=0),
      nn.LeakyReLU(0.1, inplace=True)
  )

class Reshape(nn.Module):
  def forward(self, input):
    output=input.view(-1, 1024, 4, 4)
    return output

In [ ]:
class AutoEncoder(nn.Module):
  def __init__(self):
    super(AutoEncoder, self).__init__()

    self.encoder=nn.Sequential(
        _ConvLayer(3, 128),
        _ConvLayer(128, 256),
        _ConvLayer(256, 512),
        _ConvLayer(512, 1024),
        nn.Flatten(),
        nn.Linear(1024*4*4, 1024),
        nn.Linear(1024, 1024*4*4),
        Reshape(),
        _UpScale(1024, 512)
    )

    self.decoder_A=nn.Sequential(
         _UpScale(512, 256),
         _UpScale(256, 128),
         _UpScale(128, 64),
         nn.Conv2d(64, 3, kernel_size=3, padding=1),
         nn.Sigmoid()
    )

    self.decoder_B=nn.Sequential(
         _UpScale(512, 256),
         _UpScale(256, 128),
         _UpScale(128, 64),
         nn.Conv2d(64, 3, kernel_size=3, padding=1),
         nn.Sigmoid()
    )

  def forward(self, x, select='A'):
    if select == 'A':
      out=self.encoder(x)
      out=self.decoder_A(out)
    else:
      out=self.encoder(x)
      out=self.decoder_B(out)
    return out

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary
model=AutoEncoder()
summary(model, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 32, 32]           9,728
         LeakyReLU-2          [-1, 128, 32, 32]               0
            Conv2d-3          [-1, 256, 16, 16]         819,456
         LeakyReLU-4          [-1, 256, 16, 16]               0
            Conv2d-5            [-1, 512, 8, 8]       3,277,312
         LeakyReLU-6            [-1, 512, 8, 8]               0
            Conv2d-7           [-1, 1024, 4, 4]      13,108,224
         LeakyReLU-8           [-1, 1024, 4, 4]               0
           Flatten-9                [-1, 16384]               0
           Linear-10                 [-1, 1024]      16,778,240
           Linear-11                [-1, 16384]      16,793,600
          Reshape-12           [-1, 1024, 4, 4]               0
  ConvTranspose2d-13            [-1, 512, 8, 8]       2,097,664
        LeakyReLU-14            [-1, 51